# Analyze the hybrid database

This tutorial will show you how to use the methods of the Analysis class of pyLCAIO, to perform life cycle calculations and contribution analyses.

To begin this tutorial, you need to have created your hybrid database and saved it (go check the notebook "Running_pyLCAIO"). You should therefore have a hybrid database stored in your src/Databases pyLCAIO folder.

Begin with import statements:

In [ ]:
import pandas as pd
import sys
sys.path.append('path_to_pylcaio/src/')
import pylcaio

To create an object of the Analysis you only need to provide the path to the folder containing the hybrid database.

In [ ]:
analysis_object = pylcaio.Analysis('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Databases/'+
                                   'ecoinvent3.5_exiobase3/STAM_capitals_extended_IW/hybrid_system.pickle')

Two main methods in the Analysis class:
* calc_lifecycle(), performing life cycle calculations
* contribution_analysis(), performing contribution analyses

# Life cycle calculations

In [ ]:
analysis_object.calc_lifecycle()

Calculations should take from 5 to 30 minutes depending on the calculation power of your machine and the features of the hybrid database. Results of the calculations are contained within the D matrix.

In [ ]:
analysis_object.D

analysis_object.D is a pandas dataframe, you can therefore manipulate it as any dataframe or if you are not comfortable with pandas, you can export the results to excel:

In [ ]:
analysis_object.D.to_excel('put_the_path_where_you_want_the_excel_sheet_to_be/name_of_the_excel_sheet.xlsx')

All objects of the Analysis module linked with ecoinvent work with UUIDs. To associate an ID to its features, use the PRO_f and STR matrices:

In [ ]:
metadata_economic_flows = pd.DataFrame.from_dict(analysis_object.PRO_f)
metadata_environmental_flows = pd.DataFrame.from_dict(analysis_object.STR)

# Contribution analyses

To perform contribution analyses, call the appropriate method along with the UUID of the process you wish the examine. To find the process you want, use the metadata_economic_flows dataframe we just created before.

In [ ]:
contribution_analysis = analysis_object.contributions(UUID)

If you struggle to manipulate dataframes, you can rely on the .navite_through_PRO_f() method. You can enter a product, an activity, a geography or a combination of the three to pin down the process you wish to analyze. Say you want to take a look at a process of hybroelectricity of France:

In [ ]:
analysis_object.navigate_through_PRO_f(product='electricity',geography='FR',activity='hydro')

contributions() returns multiple items. The diagonalized total demand for the selected process (X), the normalized emissions for each process (F) and the characterization matrix (C).

In [ ]:
X = contribution_analysis[0]
F = contribution_analysis[1]
C = contribution_analysis[2]

You can then execute the operation you want to either obtain total emissions or total impacts:

In [ ]:
total_emissions = F.dot(X)
total_impacts = C.dot(F).dot(X)

Note that if you a reusing a regionalized version, there are 2 additional matrices:

In [ ]:
X = contribution_analysis[0]
F = contribution_analysis[1]
C = contribution_analysis[2]
F_regio = contribution_analysis[3]
C_regio = contribution_analysis[4]

And the calculation therefore goes like this:

In [ ]:
total_emissions = F.dot(X)
total_impacts = C.dot(F).dot(X)
total_emissions_regio = F_regio.dot(X)
total_impacts_regio = C_regio.dot(F_regio).dot(X)